In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.metrics import accuracy_score

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D, Rescaling, RandomFlip, RandomBrightness, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.applications.xception import preprocess_input, Xception
from tensorflow.keras import Sequential,Model
from tensorflow.keras.optimizers import Adam

2026-01-23 09:57:36.706962: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-23 09:57:37.975948: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-23 09:57:53.537391: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
img_size = (299,299)
batch_size = 16
train_ds = keras.utils.image_dataset_from_directory(
    "train",
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary'
)

test_ds = keras.utils.image_dataset_from_directory(
    "test",
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary'
)

Found 140002 files belonging to 2 classes.


I0000 00:00:1769162287.181947   37326 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3536 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 10905 files belonging to 2 classes.


In [4]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomBrightness(0.05)
])

train_ds = train_ds.map(lambda x,y: (data_augmentation(x, training=True), y))

In [5]:
train_ds= train_ds.map(lambda x,y: (preprocess_input(x), y))
test_ds= test_ds.map(lambda x,y: (preprocess_input(x), y))

In [6]:
xception = Xception(
    weights = "imagenet",
    include_top = False,
    input_shape = (299,299,3)
)
xception.trainable = False

In [7]:
x = xception.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)

x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)

output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=xception.input, outputs=output)

In [8]:
model.compile(
    optimizer = Adam(learning_rate=1e-4),
    loss = "binary_crossentropy",
    metrics = ["accuracy", tf.keras.metrics.AUC(name = "auc")]
)

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=4,
    restore_best_weights=True
)

In [11]:
model.fit(train_ds, validation_data=test_ds, epochs=10, batch_size=32, callbacks=[early_stopping])

Epoch 1/10


2026-01-23 09:58:53.246590: I external/local_xla/xla/service/service.cc:163] XLA service 0x7399a403b180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-01-23 09:58:53.246629: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2026-01-23 09:58:53.436894: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-01-23 09:58:54.611656: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91800
2026-01-23 09:58:58.327375: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 09:58:58.469697: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel 

8750/8751 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.7213 - auc: 0.7988 - loss: 0.5570

2026-01-23 10:09:48.536955: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:09:48.673388: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:09:49.959494: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:09:50.158317: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:09:50.838629: E external/local_xla/xla/stream_

8751/8751 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.7213 - auc: 0.7988 - loss: 0.5570

2026-01-23 10:10:57.515720: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:10:57.656413: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:10:58.674339: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:10:58.856734: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-23 10:11:00.636761: E external/local_xla/xla/stream_

8751/8751 ━━━━━━━━━━━━━━━━━━━━ 742s 82ms/step - accuracy: 0.7453 - auc: 0.8274 - loss: 0.5166 - val_accuracy: 0.7146 - val_auc: 0.8008 - val_loss: 0.5742
Epoch 2/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 673s 77ms/step - accuracy: 0.7806 - auc: 0.8641 - loss: 0.4573 - val_accuracy: 0.7207 - val_auc: 0.8065 - val_loss: 0.5658
Epoch 3/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 671s 77ms/step - accuracy: 0.7921 - auc: 0.8769 - loss: 0.4366 - val_accuracy: 0.7190 - val_auc: 0.8041 - val_loss: 0.5777
Epoch 4/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 677s 76ms/step - accuracy: 0.8024 - auc: 0.8868 - loss: 0.4205 - val_accuracy: 0.7150 - val_auc: 0.7992 - val_loss: 0.6070
Epoch 5/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 673s 77ms/step - accuracy: 0.8101 - auc: 0.8938 - loss: 0.4081 - val_accuracy: 0.7179 - val_auc: 0.8019 - val_loss: 0.6085
Epoch 6/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 690s 79ms/step - accuracy: 0.8150 - auc: 0.8996 - loss: 0.3974 - val_accuracy: 0.7150 - val_auc: 0.7975 - val_loss: 0.6415


In [12]:
xception.trainable = True

for layer in xception.layers[:-30]:
    xception.trainable = False

In [13]:
model.compile(
    optimizer = Adam(learning_rate=1e-5),
    loss = "binary_crossentropy",
    metrics = ["accuracy", tf.keras.metrics.AUC(name="auc")]
)

In [15]:
model.fit(train_ds, validation_data=test_ds, epochs=10, callbacks=[early_stopping])

Epoch 1/10


8751/8751 ━━━━━━━━━━━━━━━━━━━━ 726s 83ms/step - accuracy: 0.8028 - auc: 0.8880 - loss: 0.4189 - val_accuracy: 0.7195 - val_auc: 0.8050 - val_loss: 0.5772
Epoch 2/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 708s 81ms/step - accuracy: 0.8043 - auc: 0.8894 - loss: 0.4163 - val_accuracy: 0.7186 - val_auc: 0.8047 - val_loss: 0.5794
Epoch 3/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 717s 82ms/step - accuracy: 0.8069 - auc: 0.8912 - loss: 0.4134 - val_accuracy: 0.7211 - val_auc: 0.8050 - val_loss: 0.5819
Epoch 4/10
8751/8751 ━━━━━━━━━━━━━━━━━━━━ 686s 78ms/step - accuracy: 0.8081 - auc: 0.8934 - loss: 0.4093 - val_accuracy: 0.7182 - val_auc: 0.8045 - val_loss: 0.5873


In [17]:
model.save("dfd_model.keras")